In [1]:
import os
import re
import copy
import datetime
import torch
import json
import requests
from transformers import AutoTokenizer
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)
from peft import LoraConfig, PeftModel

import transformers

from langchain.document_loaders import TextLoader
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import AsyncChromiumLoader
from langchain.document_transformers import Html2TextTransformer
from langchain.schema.runnable.passthrough import RunnablePassthrough
from langchain.llms import HuggingFacePipeline, HuggingFaceEndpoint
from langchain.prompts import PromptTemplate
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.chains import LLMChain
from langchain.docstore.document import Document

#read the environment variables from the .env file
from dotenv import dotenv_values
# print(load_dotenv())
config = dotenv_values(".env")
HUGGINGFACEHUB_API_TOKEN = config['HUGGINGFACEHUB_API_TOKEN']

model_name = "mistralai/Mistral-7B-Instruct-v0.1"

device_type = "mps"

/opt/homebrew/anaconda3/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [2]:
#################################################################
# Tokenizer
#################################################################

start_time = datetime.datetime.now()

model_config = transformers.AutoConfig.from_pretrained(
    model_name,
    device_map=device_type,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [3]:
#################################################################
# bitsandbytes parameters
#################################################################

# # Activate 4-bit precision base model loading
# use_4bit = True

# # Compute dtype for 4-bit base models
# bnb_4bit_compute_dtype = "float16"

# # Quantization type (fp4 or nf4)
# bnb_4bit_quant_type = "nf4"

# # Activate nested quantization for 4-bit base models (double quantization)
# use_nested_quant = False

# #################################################################
# # Set up quantization config
# #################################################################
# compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=use_4bit,
#     bnb_4bit_quant_type=bnb_4bit_quant_type,
#     bnb_4bit_compute_dtype=compute_dtype,
#     bnb_4bit_use_double_quant=use_nested_quant,
# )

In [4]:
#################################################################
# Load pre-trained config
#################################################################
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     # quantization_config=bnb_config,
#     device_map="auto",
# )


# def print_number_of_trainable_model_parameters(model):
#     trainable_model_params = 0
#     all_model_params = 0
#     for _, param in model.named_parameters():
#         all_model_params += param.numel()
#         if param.requires_grad:
#             trainable_model_params += param.numel()
#     return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"


# print(print_number_of_trainable_model_parameters(model))

# text_generation_pipeline = transformers.pipeline(
#     model=model,
#     tokenizer=tokenizer,
#     task="text-generation",
#     temperature=0.2,
#     repetition_penalty=1.1,
#     return_full_text=True,
#     max_new_tokens=1000,
# )

# mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline, device_map="mps")



In [58]:
prompt_template = """
### [INST] 
Instruction: Answer the question based on your 
fantasy football knowledge. Here is context to help:

{context}

### QUESTION:
{question} 

[/INST]
 """


endpoint_url = "https://api-inference.huggingface.co/models/" + model_name
#currently only ('text2text-generation', 'text-generation', 'summarization') are supporte
mistral_llm_endpoint = HuggingFaceEndpoint(endpoint_url=endpoint_url, task="text-generation", huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN)


# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain
llm_chain = LLMChain(llm=mistral_llm_endpoint, prompt=prompt)
llm_chain.__call__(inputs={})

ValueError: Missing some input keys: {'question', 'context'}

In [4]:
import nest_asyncio

nest_asyncio.apply()

# Articles to index
articles = [
    "https://www.fantasypros.com/2023/11/rival-fantasy-nfl-week-10/",
    "https://www.fantasypros.com/2023/11/5-stats-to-know-before-setting-your-fantasy-lineup-week-10/",
    "https://www.fantasypros.com/2023/11/nfl-week-10-sleeper-picks-player-predictions-2023/",
    "https://www.fantasypros.com/2023/11/nfl-dfs-week-10-stacking-advice-picks-2023-fantasy-football/",
    "https://www.fantasypros.com/2023/11/players-to-buy-low-sell-high-trade-advice-2023-fantasy-football/",
]

# Scrapes the blogs above
loader = AsyncChromiumLoader(articles)
docs = loader.load()



KeyboardInterrupt: 

In [7]:
# Create a deep copy of docs
docs_copy = copy.deepcopy(docs)

# Converts HTML to plain text
html2text = Html2TextTransformer()

_docs_transformed = html2text.transform_documents(docs_copy)

docs_transformed = []

for doc in _docs_transformed:
    try:
        metadata = copy.deepcopy(doc.metadata)
        page_content = copy.deepcopy(doc.page_content)
        page_content = page_content.split('* Apps')[1]
        page_content = page_content.replace('.\n\n', '\n~\n')
        page_content = page_content.replace('\n\n', ' ')
        page_content = page_content.replace('\n~\n', '\n\n')
        #replace individual newline characters with a space, but leave double newlines or more alone
        # doc.page_content = re.sub('\n{1,99}', ' ', doc.page_content)
        
        doc = Document(page_content=page_content, metadata=metadata)
        docs_transformed.append(doc)
    except:
        print('error')

# Chunk text
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=0)
chunked_documents = text_splitter.split_documents(docs_transformed)

# Load chunked documents into the FAISS index
db = FAISS.from_documents(
    chunked_documents,
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"),
)

retriever = db.as_retriever()

#currently only ('text2text-generation', 'text-generation', 'summarization') are supported

Created a chunk of size 558, which is longer than the specified 200
Created a chunk of size 831, which is longer than the specified 200
Created a chunk of size 830, which is longer than the specified 200
Created a chunk of size 499, which is longer than the specified 200
Created a chunk of size 1391, which is longer than the specified 200
Created a chunk of size 674, which is longer than the specified 200
Created a chunk of size 627, which is longer than the specified 200
Created a chunk of size 1165, which is longer than the specified 200
Created a chunk of size 402, which is longer than the specified 200
Created a chunk of size 286, which is longer than the specified 200
Created a chunk of size 421, which is longer than the specified 200
Created a chunk of size 470, which is longer than the specified 200
Created a chunk of size 497, which is longer than the specified 200
Created a chunk of size 577, which is longer than the specified 200
Created a chunk of size 241, which is longer t

error
error
error


In [8]:
#currently only ('text2text-generation', 'text-generation', 'summarization') are supporte


In [9]:
start_time = datetime.datetime.now()

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | llm_chain

# response = rag_chain.invoke("Which players suffered a knee injury?")
# response = rag_chain.invoke("What injury did Josh Downs suffer?")
# response = rag_chain.invoke("How many yards per game does CeeDee Lamb have?")
# response = rag_chain.invoke("What team does CeeDee Lamb play for? And what position does he play? And how many yards per game has he averaged?")
# response = rag_chain.invoke("How many touchdowns has Alexander Mattison scored?")
response = rag_chain.invoke("What team does CeeDee Lamb play for?")

print(response['text'])
print(response)

print('Total time taken:\t', datetime.datetime.now() - start_time)

 CeeDee Lamb plays for the Dallas Cowboys.
{'context': [Document(page_content='**Pick: More** ### **CeeDee Lamb (WR -DAL) : 79.5 receiving yards** CeeDee Lamb has caught fire for the Dallas Cowboys. With 117 or more receiving\nyards in three straight games, CeeDee has upped his season average to 103\nreceiving yards per game. He has seen insane volume over his last two,\naveraging 15 targets a game. CeeDee has recorded 77 or more receiving yards in\nfive of his eight contests this year and is an excellent bet to do so again\nthis weekend. Lock in the More on CeeDee for Week 10', metadata={'source': 'https://www.fantasypros.com/2023/11/nfl-week-10-sleeper-picks-player-predictions-2023/'}), Document(page_content='RB — JAC __   4. CeeDee Lamb WR — DAL   5. A.J. Brown WR — PHI   6. Jahmyr Gibbs RB — DET __   7. Tony Pollard RB — DAL   8. Alvin Kamara RB — NO __   9. Austin Ekeler RB — LAC   10. Derrick Henry RB — TEN View all Flex Rankings Fantasy football # Who Should I Start __ __ See Ad

In [10]:
# # Create a deep copy of docs
# docs_copy = copy.deepcopy(docs)

# # Converts HTML to plain text
# html2text = Html2TextTransformer()

# docs_transformed = html2text.transform_documents(docs_copy)

# for doc in docs_transformed:
#     print(doc)

print(response['text'])


 CeeDee Lamb plays for the Dallas Cowboys.


In [78]:
from langchain.vectorstores.weaviate import Weaviate
import weaviate

client = weaviate.Client("http://localhost:8080")

# vectorstore = Weaviate(client, "Upwork_jobs", "snippet")
vectorstore = Weaviate(client, "Upwork_jobs", "job_title")

retriever = vectorstore.as_retriever()

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | llm_chain


# Define the specific job description you're interested in
target_job_description = "AI, machine learning, AI/ML"

# Construct the prompt
prompt = f"Find jobs similar to: '{target_job_description}'"

response = rag_chain.invoke(prompt)
print(response)
print(response['text'])



{'context': [Document(page_content='AI/ML Specialist - Full Time/Remote'), Document(page_content='AI & ML expert to identify entry level jobs '), Document(page_content='Machine Learning for ml-cvnets for image prrocessing'), Document(page_content='AI Automation & AI Solutions')], 'question': "Find jobs similar to: 'AI, machine learning, AI/ML'", 'text': '\nBased on your fantasy football knowledge, I would suggest looking for jobs related to AI, machine learning'}

Based on your fantasy football knowledge, I would suggest looking for jobs related to AI, machine learning


In [69]:
indexes = client.schema.get()

#write indexes to json
with open('indexes.json', 'w') as outfile:
    json.dump(indexes, outfile)

# !pip show weaviate-client

In [103]:
from langchain.chat_models

client = weaviate.WeaviateClient(
        weaviate.ConnectionParams.from_url("http://localhost:8080", 50051)
    )
upwork_jobs = client.collections.get("Upwork_jobs")
response = upwork_jobs.query.near_text(
    query="machine learning snippet in the United States",
    limit=25
)

data = [json.loads(json.dumps(o.properties)) for o in response.objects]
# data = {'data': data}
# data = '\n'.join(data)



#write response to json file
with open('similarity_search.json', 'w') as outfile:
    json.dump(data, outfile, indent=2)

# for o in response.objects:
#     print(json.loads(json.dumps(o.properties)))

In [ ]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chat_models import ChatOllama

chat_model = ChatOllama(
    model="llama2:7b-chat",
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
)